In [1]:
# Fintech数据分析

In [2]:
## 量化交易

In [3]:
### Stock Buy

In [4]:
# 数据读取与简单探查

# In[1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [5]:
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")

In [6]:
df = pd.read_csv('~/data/BI/all_stocks_5yr.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

,Date,Open,High,Low,Close,Volume,name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [7]:
df['name'].nunique()
#df.info()
df.head()

,Date,Open,High,Low,Close,Volume,name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [8]:
df.name.nunique()
open_dates = np.unique(df.Date)
len(open_dates)

df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Open,High,Low,Close,Volume,name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [9]:
"""
dates = df.date.apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d', errors='ignore'))
dates
df = df.assign(day = dates) 
df
"""

df.dtypes
df.name.value_counts().head(5)

AAL     1259
NRG     1259
NOC     1259
NLSN    1259
NKE     1259
Name: name, dtype: int64

In [10]:
# 筛选苹果这支股票
sub_df = df[df.name == 'AAPL']
sub_df.head()
sub_df.plot.line(x='Date', y='Open', figsize=(15,10))

<AxesSubplot:xlabel='Date'>

In [11]:
# 计算投资组合价值
def get_portfolio_value(port, evaluation_date):
    if evaluation_date not in open_dates: 
        print('Market closed for today')    
        return 0
    # 总价值
    total_value = 0
    # 累加每支股票的value
    for stock in port.keys():
        if stock == 'cash':
            total_value += port['cash']
            continue
        # 找到evaluation_date时，该股票的price
        stock_price = df[(df.name == stock) & (df.Date == evaluation_date)].iloc[0]['Close']
        # 计算该股票的value
        position = stock_price * port[stock]
        total_value += position
    
    # 打印当前的portfolio
#     print(port)
    return total_value

In [12]:
"""
    port: 之前买过的股票
    purchase_day: 购买日期
    stock_name: 购买的股票
    num: 购买数量
"""

'\n    port: 之前买过的股票\n    purchase_day: 购买日期\n    stock_name: 购买的股票\n    num: 购买数量\n'

In [13]:
def portfolio_buy(port, purchase_day, stock_name, num):
    # 如果不开市
    if purchase_day not in open_dates: 
        print('Market closed for today')        
        return port
    # 计算股票需要购买的金额
    stock_price = df[(df.name == stock_name) & (df.Date == purchase_day)].iloc[0]['Open']
    order_price = stock_price * num
    # 考虑portfolio 钱不够的问题
    if port['cash'] < order_price:
        # 没买成，原封不动
        return port
    
    # 购买成功，cash减少
    port['cash'] -=  order_price
    # 如果之前没有持有过这个股票
    if stock_name not in port.keys():
        port[stock_name] = num
        return port
    elif stock_name in port.keys():
        port[stock_name] += num
        return port
    else:
        print('Error')
        return port

In [14]:
"""
    port: 自己手上的股票情况
    sell_day: 卖出日期
    stock_name: 股票名称
    num: 卖出数量
"""

'\n    port: 自己手上的股票情况\n    sell_day: 卖出日期\n    stock_name: 股票名称\n    num: 卖出数量\n'

In [15]:
def portfolio_sell(port, sell_day, stock_name, num):
    # 如果不开市
    if sell_day not in open_dates: 
        print('Market closed for today')
        return port    
    
    # 计算卖出的股票金额
    stock_price = df[(df.name == stock_name) & (df.Date == sell_day)].iloc[0]['Close']
    order_price = stock_price * num    
    # 如果之前没有持有过这个股票
    if stock_name not in port.keys():
        # 没卖成，原封不动
        return port
    # 如果卖出的数量 > 手上持有的数量，没卖成
    if num > port[stock_name]:
        return port    
    # 卖成功了，减少股票数量，增加cash
    if stock_name in port.keys():
        port[stock_name] -= num
        port['cash'] += order_price
        return port

In [16]:
# 假设初始资金 10000
portfolio = dict()
portfolio['cash'] = 10000


portfolio

{'cash': 10000}

In [17]:
# 买入两支股票
portfolio_buy(portfolio, '2016-01-04', 'AAPL', 10)
portfolio_buy(portfolio, '2016-01-04', 'GOOG', 10)


get_portfolio_value(portfolio, '2016-01-04')


portfolio_sell(portfolio, '2017-02-01', 'AAPL', 20)


get_portfolio_value(portfolio, '2017-02-01')

10788.35

In [18]:
# # 策略模拟
# 
# 假设我们模拟 2017-01-03 开始，初始资金 100000 元，大家各自选择购买策略, 比 2017年 最后一个交易日： 2017-12-29 的portfolio 价值。
# 
# ## 策略一： 2016-2017年，按照过去1年涨幅排序，直接全仓购买最好的
#

In [19]:
# 筛选2016-2017年的股票数据
year_2016_df = df[
    (df.Date >= pd.to_datetime('2016-01-01', format='%Y-%m-%d', errors='ignore')) &
    (df.Date <= pd.to_datetime('2017-01-01', format='%Y-%m-%d', errors='ignore')) 
]
year_2016_df


best_stock = 'Not sure yet'
growth = -1

In [20]:
for stock in list(year_2016_df.name.unique()):
    # 计算每支股票的涨幅 stock_growth
    sub_df = year_2016_df[year_2016_df.name == stock]
    open_price = sub_df[sub_df.Date == min(sub_df.Date)].iloc[0]['Open']
    close_price = sub_df[sub_df.Date == max(sub_df.Date)].iloc[0]['Open']    
    stock_growth = round(100 * (close_price - open_price) / open_price, 3)
    # print(stock, ':',  stock_growth)
    # 找到涨幅最高的股票
    if stock_growth > growth:
        best_stock = stock
        growth = stock_growth
# AMD 涨幅达到322.383%
print(best_stock, ':', growth)

AMD : 322.383


In [21]:
# 假设初始资金 10000
portfolio = dict()
portfolio['cash'] = 10000

In [22]:
# 计算能买多少股
valid_num = int(portfolio['cash'] / df[(df.name == 'AMD') & (df.Date == '2017-01-03')].iloc[0]['Open'])
valid_num

875

In [23]:
# 在2017年初 重仓AMD
portfolio_buy(portfolio, '2017-01-03', 'AMD', valid_num)

{'cash': 7.5, 'AMD': 875}

In [24]:
# 计算2017年底 投资组合value
get_portfolio_value(portfolio, '2017-12-29')

9002.5

In [25]:
# 统计2017年的交易日
trade_days_2017 = sorted(np.unique(df[
    (df.Date >= pd.to_datetime('2017-01-01', format='%Y-%m-%d', errors='ignore')) &
    (df.Date < pd.to_datetime('2018-01-01', format='%Y-%m-%d', errors='ignore'))     
]['Date'].apply(lambda x: str(x)[0:10])))
trade_days_2017

['2017-01-03',
 '2017-01-04',
 '2017-01-05',
 '2017-01-06',
 '2017-01-09',
 '2017-01-10',
 '2017-01-11',
 '2017-01-12',
 '2017-01-13',
 '2017-01-17',
 '2017-01-18',
 '2017-01-19',
 '2017-01-20',
 '2017-01-23',
 '2017-01-24',
 '2017-01-25',
 '2017-01-26',
 '2017-01-27',
 '2017-01-30',
 '2017-01-31',
 '2017-02-01',
 '2017-02-02',
 '2017-02-03',
 '2017-02-06',
 '2017-02-07',
 '2017-02-08',
 '2017-02-09',
 '2017-02-10',
 '2017-02-13',
 '2017-02-14',
 '2017-02-15',
 '2017-02-16',
 '2017-02-17',
 '2017-02-21',
 '2017-02-22',
 '2017-02-23',
 '2017-02-24',
 '2017-02-27',
 '2017-02-28',
 '2017-03-01',
 '2017-03-02',
 '2017-03-03',
 '2017-03-06',
 '2017-03-07',
 '2017-03-08',
 '2017-03-09',
 '2017-03-10',
 '2017-03-13',
 '2017-03-14',
 '2017-03-15',
 '2017-03-16',
 '2017-03-17',
 '2017-03-20',
 '2017-03-21',
 '2017-03-22',
 '2017-03-23',
 '2017-03-24',
 '2017-03-27',
 '2017-03-28',
 '2017-03-29',
 '2017-03-30',
 '2017-03-31',
 '2017-04-03',
 '2017-04-04',
 '2017-04-05',
 '2017-04-06',
 '2017-04-

In [26]:
# 计算在2017年不同交易日的value
value = []
for trading_date in trade_days_2017:
    value.append(get_portfolio_value(portfolio, trading_date))
value

port_value_line = pd.DataFrame({'d': trade_days_2017, 'value':value})

In [27]:
# 在2017年 不同交易日的value变化
port_value_line.plot.line(x = 'd', y= 'value', figsize=(15,10))

<AxesSubplot:xlabel='d'>

In [28]:
### Stock Macd

In [29]:
# 获取股票数据
import fix_yahoo_finance as yf
tickers = ['AAPL', 'BABA', 'BIDU', 'GOOGL', 'FB', 'AMZN']
data = yf.download(tickers = tickers, start = '2020-01-01', end = '2020-12-31')
data


data.to_csv("~/data/BI/macd.csv")

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/fix_yahoo_finance/__init__.py", line 322, in _download_one_threaded
    period, interval, prepost)
  File "/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/fix_yahoo_finance/__init__.py", line 333, in _download_one
    actions=actions, auto_adjust=auto_adjust)
  File "/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/fix_yahoo_finance/__in

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fix_yahoo_finance as yf

In [ ]:
# 计算df的 短期平均ma1, 长期平均ma2
def macd(df):
    # 计算MA1和MA2
    df['ma1']=df['Close'].rolling(window=ma1, min_periods=1).mean()
    df['ma2']=df['Close'].rolling(window=ma2, min_periods=1).mean()
    # 短期均线 与 长期均线的DIFF
    df['diff']=df['ma1'] - df['ma2']
    # DEA = DIFF的平滑（9天的平均值）
    df['dea']=df['diff'].rolling(window=9, min_periods=1).mean()
    return df

In [ ]:
"""
    当短均线大于长均线时，我们看多并持有
    当短均线小于长均线时，我们清仓
    背后的逻辑是短均线有动量的影响（惯性）
    我们可以用 diff = 长均线-短均线
    diff有时正，有时负
    这就是为什么称为 Moving Average Convergence Divergence
"""

In [ ]:
def signal_compute(df):
    # 计算短期平均ma1, 长期平均ma2, diff, dea
    df = macd(df)
    # 初始化positions均为0
    df['positions'] = 0

    # 当短均线 > 长均线， positions=1
    df['positions'][ma1:] = np.where(df['ma1'][ma1:]>=df['ma2'][ma1:],1,0)
    #df['positions'] = np.where(df['diff']>df['dea'], 1, 0)

    # positions 表明了需要持有，计算前后两天的positions diff，代表交易信号 signals
    # signals=1 买入，signals=-1 卖出
    df['signals'] = df['positions'].diff()

    # 震荡diff = 两个移动平均之差
    df['diff'] = df['ma1']-df['ma2']
    df['macd'] = 2 * (df['diff'] - df['dea'])
    return df

In [ ]:
# 绘制回测结果
def plot(df, ticker):    
    #the first plot is the actual close price with long/short positions
    # 绘制实际的股票收盘数据
    fig=plt.figure(figsize=(12, 6))
    ax=fig.add_subplot(111)    
    ax.plot(df.index, df['Close'], label=ticker)
    # 只显示时刻点，不显示折线图 => 设置 linewidth=0
    ax.plot(df.loc[df['signals']==1].index, df['Close'][df['signals']==1], label='Buy', linewidth=0, marker='^', c='g')
    ax.plot(df.loc[df['signals']==-1].index, df['Close'][df['signals']==-1], label='Sell', linewidth=0, marker='v', c='r')
    
    plt.legend(loc='best')
    plt.grid(True)
    plt.title('Positions')
    plt.show()
    
    # 显示diff, 即ma1-ma2
    fig=plt.figure(figsize=(12, 6))
    cx=fig.add_subplot(211)
    df['diff'].plot(kind='bar',color='r')
    #df['macd'].plot(kind='bar', color='r')
    
    plt.legend(loc='best')
    plt.grid(True)
    plt.xticks([]) # 不显示x轴刻度
    #plt.xlabel('')
    #plt.title('MACD Diff (ma1-ma2)')
    #plt.title('MACD 2*(diff-dea)')
    
    # 绘制ma1, ma2曲线
    bx=fig.add_subplot(212)
    bx.plot(df.index, df['ma1'], label='ma1')
    bx.plot(df.index, df['ma2'], label='ma2', linestyle=':')
   
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()


In [ ]:
# MACD简单有效，但是需要注意一个问题，就是：进入信号总是很晚，需要注意向下的均线
# 可以采用12,26 也可以采用 10和21
ma1 = 12
ma2 = 26

In [ ]:
# 使用slicer进行切片，如果数据很大，回测曲线会比较乱，因为有太多的标记挤在一起
slicer = 0

# 获取某一支股票的数据
data = yf.download(tickers='AAPL', start='2020-01-01', end='2020-12-31')
data.to_csv('~/data/BI/AAPL_macd.csv')
data

In [ ]:
pd.set_option('max_rows', None)
# 计算ma1, ma2, positions, signals, diff指标
df = signal_compute(data)
df.index = pd.to_datetime(df.index)
df


print('买入信号:', df.loc[df['signals']==1].index)
print('买入信号时的收盘价', df['Close'][df['signals']==1])
print('卖出信号:', df.loc[df['signals']==-1].index)

In [ ]:
plot(df, 'AAPL')

In [ ]:
# 信号的分布
df['signals'].value_counts()
df['positions'].value_counts()